# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 2 - ETL 2.1
### Natalia García Lago y Noemi Azorín

In [176]:
import requests
import pandas as pd
import numpy as np
import ast

pd.options.display.max_columns = None
from datetime import datetime, timedelta

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

In [147]:
df_ataques = pd.read_csv("../ficheros/attacks_limpieza_completa.csv", index_col = 0)
df_ataques.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.658512,-102.007710,...,1,0,0,1,0,0,2,7,4,6.313925


En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1 - El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2 - El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

In [148]:
df_info = pd.read_csv("../ficheros/ETL1.csv", index_col= 0)
df_info.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",21,-4,11,-9999,rain,2,0,45,2,Papua New Guinea
1,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",17,-4,14,-9999,rain,3,0,55,2,Papua New Guinea
2,2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,105,1,Papua New Guinea
3,3,12,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,215,2,Papua New Guinea
4,4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,260,2,Papua New Guinea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,180,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",12,2,15,-9999,rain,6,0,250,2,Papua New Guinea
60,60,183,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",11,2,15,-9999,rain,6,0,270,2,Papua New Guinea
61,61,186,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",11,2,15,-9999,rain,6,0,280,2,Papua New Guinea
62,62,189,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",12,2,15,-9999,none,6,0,270,2,Papua New Guinea


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos.

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:

    USA,
    Australia,
    New Zealand,
    South Africa,
    Papua New Guinea

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [149]:
lista_selec = ['usa', 'australia', 'papua new guinea', 'new zealand', "south africa"]

In [150]:
df_ataques_paises = df_ataques[df_ataques["country"].isin(lista_selec)]

In [151]:
#comprobamos que se ha generado un nuevo df con los paises filtrados
df_ataques_paises["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [152]:
df_info['wind_profile'] = df_info['wind_profile'].apply(ast.literal_eval)

In [153]:
df_info['rh_profile'] = df_info['rh_profile'].apply(ast.literal_eval)

In [154]:
y = df_info['wind_profile'].apply(pd.Series)

In [155]:
x = df_info['rh_profile'].apply(pd.Series)

In [156]:
df_info.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",21,-4,11,-9999,rain,2,0,45,2,Papua New Guinea
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",17,-4,14,-9999,rain,3,0,55,2,Papua New Guinea


In [157]:
for i in range(len(x.columns)):
    nombre_rh = "rh_"+ str(x[i].apply(pd.Series)["layer"][0])
    valores_rh = (x[i].apply(pd.Series)["rh"])

    df_info.insert(i, nombre_rh, valores_rh)

In [158]:
df_info.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,-9999,-9999,11,14,12,12,14,14,14,...,21,-4,11,-9999,rain,2,0,45,2,Papua New Guinea
1,-9999,-9999,-9999,13,13,12,12,14,15,15,...,17,-4,14,-9999,rain,3,0,55,2,Papua New Guinea
2,-9999,-9999,-9999,13,12,12,13,14,15,15,...,14,-1,15,-9999,rain,3,0,105,1,Papua New Guinea
3,-9999,-9999,-9999,11,12,14,15,15,15,15,...,14,-1,15,-9999,rain,3,0,215,2,Papua New Guinea
4,-9999,-9999,-9999,12,13,14,15,15,16,15,...,14,-1,15,-9999,rain,3,0,260,2,Papua New Guinea


In [160]:
for i in range(len(y.columns)):
    nombre_wind = "wind_"+ str(y[i].apply(pd.Series)["layer"][0])
    valores_wind = list(y[i].apply(pd.Series)["direction"])
    
    df_info.insert(i, nombre_wind, valores_wind)

In [161]:
df_info.head()

,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,-9999,-9999,30,30,30,35,50,80,105,...,21,-4,11,-9999,rain,2,0,45,2,Papua New Guinea
1,-9999,-9999,-9999,40,55,55,60,90,105,135,...,17,-4,14,-9999,rain,3,0,55,2,Papua New Guinea
2,-9999,-9999,-9999,10,340,330,95,105,95,75,...,14,-1,15,-9999,rain,3,0,105,1,Papua New Guinea
3,-9999,-9999,-9999,285,20,50,60,95,115,105,...,14,-1,15,-9999,rain,3,0,215,2,Papua New Guinea
4,-9999,-9999,-9999,225,345,355,5,40,105,105,...,14,-1,15,-9999,rain,3,0,260,2,Papua New Guinea


In [221]:
df_groupby = df_info.groupby("country")[df_info.columns].mean().reset_index

C:\Users\noemi\AppData\Local\Temp\ipykernel_9112\2695450577.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_groupby = df_info.groupby("country")[df_info.columns].mean().reset_index


Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.